In [16]:
import pandas as pd 

In [17]:
movies = pd.read_csv('./movielens_dataset/u.item' , header = None , sep = "|" , encoding='latin-1')
movies.columns = ['movie_id' , 'movie_title' , 'release_date' , 'video_release_date' ,
             'IMDb_URL' , 'unknown' , 'Action' , 'Adventure' , 'Animation' ,
             'Childrens' , 'Comedy' , 'Crime' , 'Documentary' , 'Drama' , 'Fantasy' ,
             'Film_Noir' , 'Horror' , 'Musical' , 'Mystery' , 'Romance' , 'Sci_Fi' ,
             'Thriller' , 'War' , 'Western']

users = pd.read_csv('./movielens_dataset/u.user', header= None , sep = '|')
users.columns = ['user_id' , 'age' , 'gender' , 'occupation' , 'zip_code']

ratings = pd.read_csv('./movielens_dataset/u.data', header= None , sep = '\t')
ratings.columns = ['user_id' , 'movie_id' , 'rating' , 'timestamp']

In [19]:
data = ratings.merge(users , on='user_id')
data = dataset.merge(movies , on='movie_id')
data

,user_id,movie_id,rating,timestamp,age,gender,occupation,zip_code,movie_title_x,release_date_x,...,Fantasy_y,Film_Noir_y,Horror_y,Musical_y,Mystery_y,Romance_y,Sci_Fi_y,Thriller_y,War_y,Western_y
0,196,242,3,881250949,49,M,writer,55105,Kolya (1996),24-Jan-1997,...,0,0,0,0,0,0,0,0,0,0
1,305,242,5,886307828,23,M,programmer,94086,Kolya (1996),24-Jan-1997,...,0,0,0,0,0,0,0,0,0,0
2,6,242,4,883268170,42,M,executive,98101,Kolya (1996),24-Jan-1997,...,0,0,0,0,0,0,0,0,0,0
3,234,242,4,891033261,60,M,retired,94702,Kolya (1996),24-Jan-1997,...,0,0,0,0,0,0,0,0,0,0
4,63,242,3,875747190,31,M,marketing,75240,Kolya (1996),24-Jan-1997,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,863,1679,3,889289491,17,M,student,60089,B. Monkey (1998),06-Feb-1998,...,0,0,0,0,0,1,0,1,0,0
99996,863,1678,1,889289570,17,M,student,60089,Mat' i syn (1997),06-Feb-1998,...,0,0,0,0,0,0,0,0,0,0
99997,863,1680,2,889289570,17,M,student,60089,Sliding Doors (1998),01-Jan-1998,...,0,0,0,0,0,1,0,0,0,0
99998,896,1681,3,887160722,28,M,writer,91505,You So Crazy (1994),01-Jan-1994,...,0,0,0,0,0,0,0,0,0,0


In [20]:
user_number_of_votes = data.groupby('user_id').size().reset_index()
user_number_of_votes.columns = ['user_id' , 'number_of_user_ratings']
data = data.merge(user_number_of_votes , on='user_id')

In [21]:
movie_number_of_votes = data.groupby('movie_id').size().reset_index()
movie_number_of_votes.columns = ['movie_id' , 'number_of_movie_ratings']
data = data.merge(movie_number_of_votes , on='movie_id')

In [22]:
user_average_rating = data.groupby('user_id')['rating'].agg('mean').reset_index()
user_average_rating.columns = ['user_id' , 'average_user_rating']
data = data.merge(user_average_rating , on='user_id')

In [23]:
movie_average_rating = data.groupby('movie_id')['rating'].agg('mean').reset_index()
movie_average_rating.columns = ['movie_id' , 'average_movie_rating']
data = data.merge(movie_average_rating , on='movie_id')

In [41]:
data['age_group'] = data['age']//10

data['total_rating'] = \
(
    data['number_of_user_ratings']
    *
    data['average_user_rating']
    + 
    data['number_of_movie_ratings']
    *
    data['average_movie_rating']
) \
/ \
(
    data['number_of_movie_ratings']
    +
    data['number_of_user_ratings']
)

data['rating_new'] = data['rating'] - data['total_rating']

In [42]:
data['rating_new']

0       -0.897436
1        1.389381
2        0.237805
3        0.706868
4       -0.604762
           ...   
99995   -0.959596
99996    1.020202
99997   -2.551282
99998    0.187500
99999   -0.977941
Name: rating_new, Length: 100000, dtype: float64